### Выбор модели
Анализировались множество популярных моделей, после первичного осмотра выявил то, что идеально подходит ```VIM-BERT```. Он обучался на научных статьях и умеет анализировать графики и диаграммы. Также обладает ещё кучей плюсов, НО оказалось что с моим слабым железом нет никакой возможности использовать его и уж тем более дообучать.  

После дальнейших изысканий поиск свёлся к моделям ```BLIP-2```, и первые два дня я мучался с ними, казалось они влезают в 12 Gb оперативки выдаваемых T4 от коллаба, НО суммарно с датасетом, распаковкой его итд не влезли.

Так что в итоге проводились эксперименты с ```BLIP-VQA-Base```. Модель из той же серии, подходящая для VQA задач.


1.1 BLIP-VQA-Base была обучена на огромных наборах данных, которые включают разнообразные изображения и текстовые описания, с акцентом на задачу визуального вопросно-ответного анализа (VQA). Эти данные позволяют модели успешно обрабатывать запросы и извлекать информацию из изображений.

1.2 Размер модели был выбран Base, чтобы наконец суметь хоть что-то проделать руками на своём ноутбуке.

1.3 Как было везде написано - BLIP-VQA-Base поддерживает простую настройку для дообучения, что важно при работе с новыми данными. Модель уже оптимизирована для задачи VQA, что упрощает её адаптацию для конкретных задач.

1.4 BLIP-VQA-Base имеет упрощенную архитектуру, которая позволяет легче проводить квантизацию и интегрировать её в производственные среды. Эта модель уже достаточно оптимизирована для быстрой работы, что упрощает её использование на разных платформах.

2.Теоретическое введение в визуально-лингвистические модели (VLM)

2.1 Архитектура VLM
Архитектура визуально-лингвистических моделей включает два основных компонента:

**Визуальный энкодер:** Обычно это свёрточные нейронные сети (CNN) или трансформеры, которые преобразуют изображения в векторные представления (эмбеддинги). Например, модели могут использовать предобученные энкодеры, такие как ResNet или ViT, для извлечения признаков изображений.

**Лингвистический энкодер:** Использует трансформеры, такие как BERT или GPT, для обработки текстов и создания текстовых эмбеддингов.

Эти два потока соединяются через механизм внимания (Attention), позволяющий модели находить взаимосвязи между визуальными и текстовыми признаками. В более сложных VLM, таких как BLIP, используется модальное выравнивание для усиления взаимодействия между визуальными и текстовыми представлениями.

2.2 Обучение VLM
Обучение VLM состоит из двух этапов:

Претрейнинг: Модель обучается на огромных наборах данных изображений с текстами, чтобы выучить общие представления. Типичные задачи включают кросс-модальные задачи, такие как сопоставление изображения и текста (Image-Text Matching), заполнение масок (Masked Language Modeling) и визуальное предсказание (Visual Prediction).

Дообучение: После претрейнинга модель может быть дообучена на специализированных данных для конкретных задач, таких как VQA, визуальная генерация текста или классификация изображений.


### Выбор датасета

После анализа были найдены несколько несложных для базового эксперимента датасетов:
- ```FigureQA```
- ```PlotQA```
- ```ChartQA (ChartMimic)```

Вопросы в FigureQA ограничены простыми бинарными ответами ("да" или "нет"), делает его проще.

PlotQA и ChartQA (ChartMimic), напротив, содержат более сложные и разнообразные вопросы, требующие глубокого анализа данных, представленных на графиках.  Они больше и было бы интересно обучить на одном и посмотреть результаты на другом. Или обучить сначала на FigureQA и смотреть прирост метрик на PlotQA потом.

Так что было принято решение начать с FigureQA и при успешных экспериментах переходить к более сложным данным.

### Код для теста модели БЕЗ дообучения

In [5]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [6]:
!pip install datasets

In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00


In [8]:
import os
from PIL import Image
import requests
import evaluate
from tqdm import tqdm
from transformers import AutoProcessor, BlipForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
from datasets import load_dataset

In [9]:
from torch.utils.data import DataLoader
import torch
import numpy as np
import io
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score

Пытался использовать модели ```blip2-flan-t5-base```, ```blip2-opt-2.7b```, но вычислительных мощностей ноутбука не хватает, как и доступного Т4 GPU в коллабе (Оперативная память системы 12.7 GB). Вылетает сразу после загрузки модели, скачивания датасета и его открытия.  
Пока у меня недостаточно опыта и ресурсов для решения таких проблем, так что была взята модель поменьше и попроще, ни а каком VIM-BERT уже речи не идёт (

In [10]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [11]:
model.eval()

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

Попробовал  chartqa - качество на метриках 0.1

In [12]:
ds = load_dataset("nimapourjafar/mm_figureqa")

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Смотрим что лежит внутри датасета и как

In [23]:
ds

DatasetDict({
    train: Dataset({
        features: ['images', 'audios', 'videos', 'data'],
        num_rows: 100000
    })
})

In [21]:
ds['train']

Dataset({
    features: ['images', 'audios', 'videos', 'data'],
    num_rows: 100000
})

In [22]:
ds['train']['images'][0]

[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\x08\x00\x00\x01\x90\x08\x06\x00\x00\x00\n\xafG\x10\x00\x00?\x1eIDATx\x9c\xed\xddyXUu\xe2\xc7\xf1\xcfa\x15\\A\x11\xc1\x15\x15\x05q\x19\x97\xc4\x16\xcc4\xd36-\x1f\'\xd3\x1a\xb3\xa6\x9a\x99\x9a1\x97\xb4t\xaa+\xce\x98\x99\xadj\xa5fS\xe9\xd8\xe2\xd4\xb4\xd7T\xa4F\xa6f\x99\x9a\x88\x12j\xb8]E\x05Tv\x04\xee\xef\x0f\x9f\xce/\x945\xe0\x1e\xce\xed\xfdz\x9e\xfb\xd4=\xe7^\xf8\xf8<\x87{\xef\xe7\x9e\xef\xf9~\r\x97\xcbu\xb9\xa4!\x02\x00\x00\x00\xf0\x9b\xe7\xa3s\xe5`\x8e\xb51\x00\x00\x00\x004\x04^V\x07\x00\x00\x00\x00\xd0pP\x10\x00\x00\x00\x00\x98(\x08\x00\x00\x00\x00L\x14\x04\x00\x00\x00\x00&\n\x02\x00\x00\x00\x00\x13\x05\x01\x00\x00\x00\x80\x89\x82\x00\x00\x00\x00\xc0DA\x00\x00\x00\x00`\xa2 \x00\x00\x00\x000Q\x10\x00\x00\x00\x00\x98<\xbe \x9c:uJ\xb7\xddv\x9b&O\x9e\xac\x92\x92\x92\n\x1fWXX\xa8\xbf\xfd\xedo\x1a>|\xb8\x86\x0f\x1f\xae\xb4\xb44\xf7\x85\x04\x00\x00\x00\x1a\x08\x8f)\x08?\xfe\xf8\xa3\xf9\xe1>??\xdf\xdc~\xec\xd819\x9dN\xfd\xf8\xe3\x

In [20]:
dict(ds)['train']['data'][0]

[{'data': '0', 'modality': 'image', 'role': 'user'},
 {'data': 'Is Pale Green the minimum?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'No.', 'modality': 'text', 'role': 'assistant'},
 {'data': 'Is Dark Red the maximum?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'No.', 'modality': 'text', 'role': 'assistant'},
 {'data': 'Is Pale Green greater than Dark Red?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'Yes.', 'modality': 'text', 'role': 'assistant'},
 {'data': 'Is Dark Red less than Pale Green?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'Yes.', 'modality': 'text', 'role': 'assistant'},
 {'data': 'Is Dark Red greater than Pale Green?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'No.', 'modality': 'text', 'role': 'assistant'},
 {'data': 'Is Pale Green less than Dark Red?\nAnswer yes or no.',
  'modality': 'text',
  'role': 'user'},
 {'data': 'No.', '

Код для получения предсказаний

In [30]:
def process_sample(sample, processor, model):
    image = Image.open(io.BytesIO(sample['images'][0]['bytes']))

    qa_pairs = [(item['data'], next(iter(sample['data'][i+1:]))['data'])
                for i, item in enumerate(sample['data'])
                if item['role'] == 'user' and item['modality'] == 'text']

    results = []
    for question, true_answer in qa_pairs:
        inputs = processor(images=image, text=question, return_tensors="pt")

        with torch.no_grad():
            outputs = model.generate(**inputs)

        predicted_answer = processor.decode(outputs[0], skip_special_tokens=True)

        results.append({
            'question': question,
            'true_answer': true_answer.lower(),
            'predicted_answer': predicted_answer.lower()
        })

    return results

def make_predictions(dataset, processor, model, num_samples=100):
    all_results = []

    for sample in tqdm(dataset['train'].select(range(num_samples))):
        results = process_sample(sample, processor, model)
        all_results.extend(results)

    return all_results

# Функция для вычисления EM метрики
def compute_exact_match(true_answer, predicted_answer):
    return int(true_answer == predicted_answer)

# Функция для вычисления F1 score
def compute_f1_score(true_answer, predicted_answer):
    true_tokens = set(true_answer.split())
    pred_tokens = set(predicted_answer.split())

    common_tokens = true_tokens.intersection(pred_tokens)

    if len(true_tokens) == 0 and len(pred_tokens) == 0:
        return 1.0

    precision = len(common_tokens) / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = len(common_tokens) / len(true_tokens) if len(true_tokens) > 0 else 0

    if precision + recall == 0:
        return 0.0

    f1 = 2 * precision * recall / (precision + recall)
    return f1



# Делаем предсказания
predictions = make_predictions(ds, processor, model, num_samples=50) # взяли 50 первых объектов, в них итак в каждом по 8-10 вопросов

100%|██████████| 50/50 [26:33<00:00, 31.86s/it]


In [79]:
predictions[1:6]

[{'question': 'Is Dark Red the maximum?\nAnswer yes or no.',
  'true_answer': 'no.',
  'predicted_answer': 'yes'},
 {'question': 'Is Pale Green greater than Dark Red?\nAnswer yes or no.',
  'true_answer': 'yes.',
  'predicted_answer': 'equal'},
 {'question': 'Is Dark Red less than Pale Green?\nAnswer yes or no.',
  'true_answer': 'yes.',
  'predicted_answer': 'yes'},
 {'question': 'Is Dark Red greater than Pale Green?\nAnswer yes or no.',
  'true_answer': 'no.',
  'predicted_answer': 'equal'},
 {'question': 'Is Pale Green less than Dark Red?\nAnswer yes or no.',
  'true_answer': 'no.',
  'predicted_answer': 'yes'}]

Увидели то что модель не очень понимает формат ответа который от неё ждут, пооэтому чуть подправили ответы (при подсчёте метрик добавляем точку в конец чтобы совпдали предсказания)

In [33]:
def compute_metrics(predictions):
    metrics = defaultdict(list)

    for pred in predictions:
        true_answer = pred['true_answer']
        predicted_answer = pred['predicted_answer']+'.'

        metrics['accuracy'].append(int(true_answer == predicted_answer))
        metrics['em'].append(compute_exact_match(true_answer, predicted_answer))
        metrics['f1'].append(compute_f1_score(true_answer, predicted_answer))

    final_metrics = {
        'accuracy': np.mean(metrics['accuracy']),
        'em': np.mean(metrics['em']),
        'f1': np.mean(metrics['f1'])
    }

    return final_metrics

In [34]:
# Вычисляем метрики
metrics = compute_metrics(predictions)

print("Metrics:")
for metric_name, metric_value in metrics.items():
    print(f"{metric_name.capitalize()}: {metric_value:.4f}")

Metrics:
Accuracy: 0.2504
Em: 0.2504
F1: 0.2504


В целом неплохо учитывая что данные и вопросы модель видит впервые и даже не понимает в каком формате ждут ответ

### Дообучение

In [80]:
class FigureQADataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = Image.open(io.BytesIO(sample['images'][0]['bytes']))

        question = sample['data'][1]['data']
        answer = sample['data'][2]['data']

        inputs = self.processor(images=image, text=question, return_tensors="pt", padding="max_length", truncation=True)

        inputs['labels'] = self.processor(text=answer, return_tensors="pt", padding="max_length", truncation=True).input_ids.squeeze()
        return inputs

train_dataset = FigureQADataset(ds['train'].select(range(100, 1100)), processor)
val_dataset = FigureQADataset(ds['train'].select(range(100)), processor)

In [81]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True
)

# Определение тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.tokenizer,
    data_collator=default_data_collator
)

# Запуск обучения
trainer.train()


ValueError: too many values to unpack (expected 4)

Казалось бы всё шло отлично, но тут случилось нечто ужасное:
В один момент у меня работало дообучение, НО было очень долгим даже для 1000 объектов, и, соответственно, не давало ожидаемого прироста в качестве:
Понятно же что отвечать хуже чем случайное гадание на задаче где ответы да и нет - это очень плохо
Так что надежда была на то что модель поймёт ожидаемый выход ‘yes.’ или ‘no.’ И будет отвечать хотя бы на 60-70% даже если она саму задачу решает плохо.

Но пока у меня работало дообучение, оно почти не улучшало метрики, а когда я начал менять схему, ища возможные другие способы дообучения, то всё сломалось и мне так и не удалось починить


Считаю, что бесценный опыт в любом случае получен, наверняка не предполагалось что кандидат на роль стажёра/Джуна идеально решит столь небазовую задачу с такими ограничениями и без подсказок….
так что можно формулировать результаты:

### Заключение



*   Разобрался с тем что такое VLM и их теоретическим устройством.
*   Изучил передовые VLM модели и их отличия.
*   Выбрал несколько моделей подходящих для решения поставленной задачи по ряду критериев.
*   Выбрал из них модель, возможности которой смог опробовать при своих ограничениях по железу.
*   Проанализировал множество датасетов и нашёл наиболее подходящие для конкретной задачи, выделил тот на котором удалось получить качество до дообучения больше 0 .
*   Запустил модель на датасете и, проанализировав результаты, получил значения метрик.
*   Разобрался с процессом дообучения и попытался запустить его со своей моделью, датасетом и железом.

